In [4]:
%cd ../

e:\python\atomization-energy-regression


In [5]:
from src.data_processing.molecule import get_molecule_name 
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error

In [6]:
data = scipy.io.loadmat('dataset\qm7.mat')
X = data['X'] # Coulomb matrices -> 7165 x 23 x 23
T = data['T'].T.squeeze() # atomization energies Y -> 7165 x 1
Z = data['Z'] # atomic charge -> 7165 x 23
R = data['R'] # cartesian coordinates -> 7165 x 23 x 3
data_train, data_test = {}, {}
molecule_name = get_molecule_name(data)


In [60]:
v =data['X'][5577]
np.linalg.norm(v, axis=1)
idx = np.argsort(np.linalg.norm(v, axis=1))
v = v[idx,:]
np.linalg.eigvals(v)

array([ 0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        , 11.350574  +0.j        ,
       -2.685962  +0.j        ,  2.1595643 +0.j        ,
       -1.3413086 +0.j        ,  0.9704444 +0.j        ,
        0.25353876+0.j        ,  0.16964687+0.20988576j,
        0.16964687-0.20988576j, -0.2560594 +0.13355258j,
       -0.2560594 -0.13355258j, -0.2869199 +0.j        ,
       -0.06175096+0.21241368j, -0.06175096-0.21241368j,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ], dtype=complex64)

In [68]:
def feature_engineer(data):
    X = data['X'] # Coulomb matrices -> 7165 x 23 x 23
    T = data['T'].T.squeeze() # atomization energies Y -> 7165 x 1
    Z = data['Z'] # atomic charge -> 7165 x 23
    R = data['R'] # cartesian coordinates -> 7165 x 23 x 3
    data_train, data_test = {}, {}
    molecule_name = get_molecule_name(data)
    
    
    y = np.transpose(T)
    y_scaling_factor = np.max(np.absolute(y))
    y_scaled = y/y_scaling_factor
   
    features_vector = []
    for (x,z,r) in zip(X,Z,R):
        sorted_idx = np.argsort(np.linalg.norm(x, axis=1)) 
        sorted_coulomb_mat = x[sorted_idx, :]  # Sort rows
        sorted_coulomb_mat.sort(axis=1)
        order_x = sorted_coulomb_mat
        features_vector.append(np.concatenate((np.linalg.eigvals(x), list(nx.degree_centrality(nx.from_numpy_matrix(x)).values()), order_x.flatten(),z, r.mean(axis=0), r.std(axis=0))))
        # print(features_vector[-1].shape)
    return features_vector, y_scaled, y_scaling_factor

X, Y, min_max_scaler  = feature_engineer(data)
X = np.asarray(X)
Y = np.asarray(Y)

In [8]:
X[0].shape

(575,)

In [69]:
def cross_validation(X, y, model, splits):
    mae = []
    for (idx, split) in enumerate(splits):
        mask = np.zeros(y.size, dtype=bool)
        mask[split] = True
        X_train = X[~mask]
        y_train = y[~mask]
        X_test = X[mask]
        y_test = y[mask]
        
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mae.append(mean_absolute_error(y_pred, y_test))
        print(mae[-1])
    return mae
from sklearn.kernel_ridge import KernelRidge
krr_model = KernelRidge(alpha = 0.0001, kernel = "rbf", gamma=0.0001)



mae = cross_validation(X, Y, krr_model, data['P'])
mae = [x * min_max_scaler for x in mae]
print(np.mean(mae))

0.002145205311624165
0.0022841842393681454
0.0021534784768279113
0.002244288384315195
0.0022152076449769207
4.840972402638049


In [ ]:
7.828280902462109

In [71]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, mutual_info_regression
X_new = SelectKBest(f_regression, k=550).fit_transform(X, Y)
mae = cross_validation(X_new, Y, krr_model, data['P'])
mae = [x * min_max_scaler for x in mae]
print(np.mean(mae))

0.002358472093598374
0.0024254944526359685
0.002316148727643307
0.002479579484810037
0.002495159201691131
5.293615976230074


In [12]:
X_new.shape

(7165, 200)

In [72]:
from sklearn.svm import SVR
svr_model = SVR(kernel='rbf', gamma=1e-4, epsilon=1e-6)


mae = cross_validation(X, Y, svr_model, data['P'])
mae = [x * min_max_scaler for x in mae]
print(np.mean(mae))

0.0028786168777998828
0.0029050120220716416
0.0027755819021060913
0.0030067621560669567
0.002783547730923894
6.290829870043775


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
kernel = DotProduct() + WhiteKernel()
gpr_model = GaussianProcessRegressor(kernel=kernel, random_state=0)
mae = cross_validation(X, Y, gpr_model, data['P'])
mae = [x * min_max_scaler for x in mae]
print(np.mean(mae))

In [75]:
np.mean(mae)

9.005196275938669

In [35]:
krr_model.fit(X, Y)

KernelRidge(alpha=0.0001, coef0=1, degree=3, gamma=0.0001, kernel='rbf',
            kernel_params=None)

In [36]:
krr_model.score(X, Y )

0.9999845458858015

In [90]:
V = {
    'X':1,
    'Y':2,
    'Z': 3
}
t1, t2, y3 = V
t1

'X'